<a href="https://colab.research.google.com/github/hkdan502/NLP/blob/main/ner_train_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 패키지 설치하기

TPU 관련 패키지를 설치합니다. TPU 사용시 아래 라인 첫 문자(#)를 지우고 수행하세요. GPU를 쓴다면 아래 라인을 실행할 필요가 없습니다.

In [27]:
# TPU 사용시 아래 라인 첫 문자(#)를 지우고 수행하세요.
#!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl

TPU 이외에 의존성 있는 패키지를 설치합니다.

In [28]:
!pip install ratsnlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# 구글 드라이브 연동하기
모델 체크포인트 등을 저장해 둘 구글 드라이브를 연결합니다. 자신의 구글 계정에 적용됩니다.

In [29]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


# 각종 설정
모델 하이퍼파라메터(hyperparameter)와 저장 위치 등 설정 정보를 선언합니다.

In [30]:
import torch
from ratsnlp.nlpbook.ner import NERTrainArguments
args = NERTrainArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_corpus_name="ner",
    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-ner",
    batch_size=32 if torch.cuda.is_available() else 4,
    learning_rate=5e-5,
    max_seq_length=64,
    epochs=3,
    tpu_cores=0 if torch.cuda.is_available() else 8,
    seed=7,
)

# 랜덤 시드 고정
학습 재현을 위해 랜덤 시드를 고정합니다.

In [31]:
from ratsnlp import nlpbook
nlpbook.set_seed(args)

set seed: 7


# 로거 설정
메세지 출력 등을 위한 logger를 설정합니다.

In [32]:
nlpbook.set_logger(args)

INFO:ratsnlp:Training/evaluation parameters NERTrainArguments(pretrained_model_name='beomi/kcbert-base', downstream_task_name='named-entity-recognition', downstream_corpus_name='ner', downstream_corpus_root_dir='/content/Korpora', downstream_model_dir='/gdrive/My Drive/nlpbook/checkpoint-ner', max_seq_length=64, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=3, batch_size=32, cpu_workers=2, fp16=False, tpu_cores=0)
INFO:ratsnlp:Training/evaluation parameters NERTrainArguments(pretrained_model_name='beomi/kcbert-base', downstream_task_name='named-entity-recognition', downstream_corpus_name='ner', downstream_corpus_root_dir='/content/Korpora', downstream_model_dir='/gdrive/My Drive/nlpbook/checkpoint-ner', max_seq_length=64, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=3, batch_size=32, cpu_workers=2, fp16=Fals

# 말뭉치 다운로드
실습에 사용할 말뭉치(NER)를 다운로드합니다.

In [33]:
nlpbook.download_downstream_dataset(args)

INFO:ratsnlp:cache file(/content/Korpora/ner/train.txt) exists, using cache!
INFO:ratsnlp:cache file(/content/Korpora/ner/train.txt) exists, using cache!
INFO:ratsnlp:cache file(/content/Korpora/ner/train.txt) exists, using cache!
INFO:ratsnlp:cache file(/content/Korpora/ner/val.txt) exists, using cache!
INFO:ratsnlp:cache file(/content/Korpora/ner/val.txt) exists, using cache!
INFO:ratsnlp:cache file(/content/Korpora/ner/val.txt) exists, using cache!


# 토크나이저 준비
토큰화를 수행하는 토크나이저를 선언합니다

In [34]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

# 학습데이터 구축
학습데이터를 만듭니다.

In [35]:
from ratsnlp.nlpbook.ner import NERCorpus, NERDataset
from torch.utils.data import DataLoader, SequentialSampler, RandomSampler

#NERCorpus 클래스는 텍스트 파일을 '원본 문장 + 개체명 태그 레이블한 문장' 형태로 읽는다.
#NERDataset 클래스는 원본 문장(text)을 토큰화한다. 원본 문장의 토큰 시퀀스 앞뒤에 CLS, SEP을 붙이고 max_seq_length(64)가 되도록 패딩 토큰을 뒤게 추가한다.
corpus = NERCorpus(args)
train_dataset = NERDataset(
    args=args,
    corpus=corpus, #corpus 데이터(원본 문장, 레이블한 문장)를 모델이 학습할 수 있는 형태(NER Features)로 변환
    tokenizer=tokenizer, #토크나이저
    mode="train",
)
train_dataloader = DataLoader(
    train_dataset,
    batch_size=args.batch_size,
    sampler=RandomSampler(train_dataset, replacement=False),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

INFO:ratsnlp:Loading features from cached file /content/Korpora/ner/cached_train_BertTokenizer_64_ner_named-entity-recognition [took 3.338 s]
INFO:ratsnlp:Loading features from cached file /content/Korpora/ner/cached_train_BertTokenizer_64_ner_named-entity-recognition [took 3.338 s]
INFO:ratsnlp:Loading features from cached file /content/Korpora/ner/cached_train_BertTokenizer_64_ner_named-entity-recognition [took 3.338 s]


In [36]:
corpus

In [37]:
train_dataset

In [38]:
train_dataset[3]

NERFeatures(input_ids=[2, 1, 3476, 4153, 2270, 4042, 420, 4185, 4346, 11, 11524, 12, 197, 9250, 11662, 8805, 8217, 17, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], token_type_ids=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], label_ids=[0, 4, 5, 15, 4, 4, 5, 15, 15, 4, 6, 4, 4, 4, 4, 4, 4, 4, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

- input_ids : 인덱스로 변환된 토큰 시퀀스
- attention_mask : 해당 토큰이 패딩 토큰(0)인지 아닌지(1) 즉, 실제 데이터면 1
- token_type_ids : 세그먼트정보, 모두 0
- label_ids : 토큰별 각 개체명 태그(B-PER, I-PER 등)를 정수로 바꾼 결과

In [39]:
train_dataset[0]

NERFeatures(input_ids=[2, 10704, 2287, 7965, 10598, 4327, 10819, 4042, 11790, 17431, 3354, 16729, 4679, 917, 1530, 1801, 9678, 359, 4443, 23831, 4062, 9511, 2451, 5953, 4034, 2173, 19033, 19778, 1898, 4110, 29483, 721, 4043, 10327, 788, 8517, 9212, 17, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], token_type_ids=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], label_ids=[0, 4, 4, 4, 4, 4, 5, 4, 4, 4, 6, 16, 4, 6, 16, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

# 테스트 데이터 구축
학습 중에 평가할 테스트 데이터를 구축합니다.

In [40]:
val_dataset = NERDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="val",
)
val_dataloader = DataLoader(
    val_dataset,
    batch_size=args.batch_size,
    sampler=SequentialSampler(val_dataset), #
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)


INFO:ratsnlp:Loading features from cached file /content/Korpora/ner/cached_val_BertTokenizer_64_ner_named-entity-recognition [took 0.102 s]
INFO:ratsnlp:Loading features from cached file /content/Korpora/ner/cached_val_BertTokenizer_64_ner_named-entity-recognition [took 0.102 s]
INFO:ratsnlp:Loading features from cached file /content/Korpora/ner/cached_val_BertTokenizer_64_ner_named-entity-recognition [took 0.102 s]


# 모델 초기화
프리트레인이 완료된 BERT 모델을 읽고, 개체명 인식 수행할 모델을 초기화합니다.

In [41]:
from transformers import BertConfig
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=corpus.num_labels,
)

In [42]:
from transformers import BertForTokenClassification
model = BertForTokenClassification.from_pretrained(
        args.pretrained_model_name,
        config=pretrained_model_config,
)

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the

# 학습 준비
Task와 Trainer를 준비합니다.

In [43]:
from ratsnlp.nlpbook.ner import NERTask
# optimizer(:Adam), learning rate scheduler(:ExponentialLR)
task = NERTask(model, args)

In [44]:
trainer = nlpbook.get_trainer(args)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True, used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


# 학습
준비한 데이터와 모델로 학습을 시작합니다. 학습 결과물(체크포인트)은 미리 연동해둔 구글 드라이브의 준비된 위치(`/gdrive/My Drive/nlpbook/checkpoint-ner`)에 저장됩니다.

In [45]:
trainer.fit(
    task,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
)

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:608: UserWarning: Checkpoint directory /gdrive/My Drive/nlpbook/checkpoint-ner exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/core/optimizer.py:380: RuntimeWarning: Found unsupported keys in the optimizer configuration: {'scheduler'}
  rank_zero_warn(
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | BertForTokenClassification | 108 M 
-----------------------------------------------------
108 M     Trainable params
0         Non-trainable params
108 M     Total params
433.389   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]